# Kiểm thử domain randomization cho FlyThruGate

Notebook này kiểm tra nhanh việc random hóa động lực học (khối lượng, lực đẩy, drag) và gió trong `FlyThrugateNewAviary.FlyThruGateAvitary`. Chạy tuần tự các cell để xem giá trị được lấy mẫu mỗi lần reset và tác động gió lên drift khi hover.

In [ ]:
import numpy as np
from pprint import pprint

from gym_pybullet_drones.envs.FlyThrugateNewAviary import FlyThruGateAvitary
from gym_pybullet_drones.utils.enums import ObservationType, ActionType

In [ ]:
def collect_samples(n_resets=10, use_randomization=True):
    """Trả về list thông tin domain_randomization sau nhiều lần reset."""
    env = FlyThruGateAvitary(
        obs=ObservationType.KIN,
        act=ActionType.RPM,
        gui=False,
        use_domain_randomization=use_randomization,
    )
    samples = []
    for _ in range(n_resets):
        _, info = env.reset()
        samples.append(info.get("domain_randomization", {}))
    env.close()
    return samples


def summarize(samples):
    """In min/max/mean cho các giá trị scalar trong domain_randomization."""
    if not samples:
        print("Không có mẫu để thống kê")
        return
    keys = [k for k, v in samples[0].items() if not isinstance(v, (list, tuple, np.ndarray))]
    for key in keys:
        arr = np.array([s[key] for s in samples], dtype=float)
        print(f"{key:15s} min={arr.min():.3f} max={arr.max():.3f} mean={arr.mean():.3f}")


def rollout_hover(steps=180, use_randomization=True):
    """Hover với action zero và đo drift để cảm nhận gió/drag ngẫu nhiên."""
    env = FlyThruGateAvitary(
        obs=ObservationType.KIN,
        act=ActionType.RPM,
        gui=False,
        use_domain_randomization=use_randomization,
    )
    _, info = env.reset()
    zero_action = np.zeros_like(env.action_space.sample())
    positions = []
    winds = []
    for _ in range(steps):
        _, _, terminated, truncated, info = env.step(zero_action)
        positions.append(info["position"])
        winds.append(info["wind_vector"])
        if terminated or truncated:
            break
    env.close()
    return np.array(positions), np.array(winds)


def drift(positions):
    if len(positions) == 0:
        return np.nan
    return float(np.linalg.norm(positions[-1] - positions[0]))

In [ ]:
# 1) Lấy mẫu ngẫu nhiên nhiều lần reset và so sánh khi bật/tắt domain randomization
rand_samples = collect_samples(n_resets=8, use_randomization=True)
print("Domain randomization ON — thống kê:")
summarize(rand_samples)
print("Ví dụ wind vector (3 lần reset đầu):")
for sample in rand_samples[:3]:
    print(sample["wind_vector"])

base_samples = collect_samples(n_resets=2, use_randomization=False)
print("\nDomain randomization OFF — nên cố định:")
pprint(base_samples[0])

In [ ]:
# 2) Hover với action zero để xem drift do gió/drag
pos_rand, winds_rand = rollout_hover(steps=240, use_randomization=True)
pos_nom, winds_nom = rollout_hover(steps=240, use_randomization=False)

print(f"Randomized drift ( {len(pos_rand)} steps ): {drift(pos_rand):.3f} m")
print(f"Nominal drift    ( {len(pos_nom)} steps ): {drift(pos_nom):.3f} m")
print("Wind step 1 (randomized):", winds_rand[0] if len(winds_rand) else None)
print("Wind step 1 (nominal):   ", winds_nom[0] if len(winds_nom) else None)